In [ ]:
from __future__ import print_function, division
import numpy as np
import h5py
import scipy.io
import random
import sys,os
import itertools
import numbers
from collections import Counter
from warnings import warn
from abc import ABCMeta, abstractmethod
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import gc
import seaborn as sns
import pandas as pd


In [ ]:
import tensorflow as tf
import pysster

In [ ]:
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(1337)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
#python_random.seed(1337)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
#tf.random.set_seed(1337)
#older version of tensorflow
tf.compat.v1.set_random_seed(1337)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
gpus = tf.compat.v1.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.compat.v1.config.experimental.set_memory_growth(gpu, True)

In [ ]:
num_threads = 8
# Maximum number of threads to use for OpenMP parallel regions.
os.environ["OMP_NUM_THREADS"] = "8"
# Without setting below 2 environment variables, it didn't work for me. Thanks to @cjw85 
os.environ["TF_NUM_INTRAOP_THREADS"] = "8"
os.environ["TF_NUM_INTEROP_THREADS"] = "8"

tf.compat.v1.config.threading.set_inter_op_parallelism_threads(
    num_threads
)
tf.compat.v1.config.threading.set_intra_op_parallelism_threads(
    num_threads
)
tf.compat.v1.config.set_soft_device_placement(True)

In [ ]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.regularizers import (
    l2, 
    l1, 
    l1_l2
)
#from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import (
    activations, 
    initializers, 
    regularizers, 
    constraints
)

In [ ]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping
)
from sklearn.metrics import (
    roc_curve,
    auc,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample, shuffle
from sklearn.feature_selection import (
    SelectKBest,
    chi2
)
from collections import defaultdict

In [ ]:
from attention_layer import Attention, attention_flatten

In [ ]:
def build_model():
	print('building model')

	seq_input_shape = (1000,4)
	nb_filter = 256
	filter_length = 9
	attentionhidden = 256

	seq_input = Input(shape = seq_input_shape, name = 'seq_input')
	convul1   = Convolution1D(filters = nb_filter,
                        	  kernel_size = filter_length,
                        	  padding = 'valid',
                        	  activation = 'relu',
                        	  kernel_constraint = MaxNorm(3), subsample_length = 1
                        	  )

	pool_ma1 = MaxPooling1D(pool_size = 3)
	dropout1 = Dropout(0.5977908689086315)
	dropout2 = Dropout(0.50131233477637737)
	decoder  = Attention(hidden = attentionhidden, activation = 'linear')
	dense1   = Dense(1)
	dense2   = Dense(1)

	output_1 = pool_ma1(convul1(seq_input))
	output_2 = dropout1(output_1)
	att_decoder  = decoder(output_2)
	output_3 = attention_flatten(output_2.shape[2])(att_decoder)

	output_4 =  dense1(dropout2(Flatten()(output_2)))
	all_outp =  concatenate([output_3, output_4])
	output_5 =  dense2(all_outp)
	output_f =  Activation('sigmoid')(output_5)

	model = Model(inputs = seq_input, outputs = output_f)
	model.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate=1e-3), metrics = ['accuracy'])

	#print(model.summary())
	return model


In [ ]:
def data_processing():
    data = pysster.data.Data(['data/VIS_pos_final.bed.fasta', 'data/VIS_neg_final.bed.fasta'], ('ACTU'))
    data.train_val_test_split(0.8, 0.1, seed = 1337)
    print(data.getsummary())

    ##test 1:1
    neg_val = np.where(valy == 0)
    pos_val = np.where(valy == 1)
    xval_positive = valx[pos_val]
    yval_positive = valy[pos_val]
    xval_negative = valx[neg_val]
    yval_negative = valy[neg_val]

    permutation = np.random.permutation(xval_negative.shape[0])
    xval_negative_1 = xval_negative[permutation[:xval_positive.shape[0]], :, :]
    yval_negative_1 = yval_negative[permutation[:xval_positive.shape[0]]]

    xval_1 = np.concatenate((xval_positive, xval_negative_1), axis=0)
    yval_1 = np.concatenate((yval_positive, yval_negative_1), axis=0)
    xval_2, yval_2 = shuffle(xval_1, yval_1 #, random_state=42
                            )

    trainx2, trainy2 = shuffle(trainx, trainy #, random_state = 42
                            )
    
    return trainx2, trainy2, xval_2, yval_2

In [ ]:
data_p = 'data/VIS_pos_final.bed.fasta'
data_n = 'data/VIS_neg_final.bed.fasta'
seq = open(data_p)
lines = seq.readlines()
seq.close()
y_p = []
chr_p = []
for line in lines:
    line = line.strip()
    if line[0] == '>':
        chr_p.append(line)
    else:
        y_p.append(line)

In [ ]:
seq_n = open(data_n)
lines_n = seq_n.readlines()
seq_n.close()
y_n = []
chr_n = []
for line_n in lines_n:
    line_n = line_n.strip()
    if line_n[0] == '>':
        chr_n.append(line_n)
    else:
        y_n.append(line_n)

In [ ]:
pos_index = range(len(y_p))
neg_index = range(len(y_n))

In [ ]:
permutation = np.random.permutation(len(neg_index))

In [ ]:
plength = len(y_p)

In [ ]:
y_n1 = []
chr_n1 = []
for num in permutation:
    n = y_n[num]
    chr = chr_n[num]
    y_n1.append(n)
    chr_n1.append(chr)

In [ ]:
u = range(10)
u1 = []
[u1.append(x) for x in u]
perm_u = np.random.permutation(u1)
u2 = perm_u[0]
u3 = u2+1
plength = len(pos_index)
#if u2 != 9:
#    y_n1 = y_n[permutation[u2*plength]:permutation[(u3*plength)]]
#    chr_n1 = chr_n[permutation[u2*plength]:permutation[(u3*plength)]]
#else:
#    y_n1 = y_n[permutation[u2*plength]:]
#    chr_n1 = chr_n[permutation[u2*plength]:]

In [ ]:
y_n2 = y_n1[u2*plength:(u2*plength+plength)]
chr_n2 = chr_n1[u2*plength:(u2*plength+plength)]

In [ ]:
print(y_n2[0])
print(chr_n2[0])

In [ ]:
print(u2)
print(permutation[u2*plength])
print(permutation[u2*plength+plength])

In [ ]:
permutation[4*31878]

In [ ]:
print(y_n[216378])
print(chr_n[216378])

In [ ]:
print(len(y_n2))
print(len(chr_n2))

In [ ]:
ofile = open('data/VIS_neg_balancedsample.fasta', 'w')
for i in range(len(y_n2)):
    ofile.write(chr_n2[i]+'\n'+y_n2[i]+'\n')
ofile.close()

In [ ]:
#def motif_analysis():
output_folder = 'pysster_motifanalysis/'
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

In [ ]:
from pysster.Data import Data

In [ ]:
pos_seq = 'data/VIS_pos_final.bed.fasta'
neg_seq = 'data/VIS_neg_balancedsample.fasta'

In [ ]:
data = Data([pos_seq, neg_seq], ('ACTU'))
data.train_val_test_split(0.8, 0.1, seed = 1337)
print(data.get_summary())

In [ ]:
params = {'conv_num':1, 'kernel_num':256, 'kernel_len':9, 'pool_size':3, 'pool_stride':1,
            'dense_num':2, 'neuron_num': 10, 'dropout_input':0, 'dropout_conv':0.5, 'dropout_dense':0.5,
            'learning_rate':1e-3, 'epochs':100}

In [ ]:
from pysster.Model import Model
from pysster import utils

In [ ]:
model = Model(params, data)

In [ ]:
model.train(data, verbose=True)